In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt


# Import necessary dependecies
from pathlib import Path
from lightglue import LightGlue, LightGlue_custom, SuperPoint, SuperGlue, SIFT
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch

torch.set_grad_enabled(False)


In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape[0], img1.shape[1]
    draw_size1 = int(np.max([np.ceil(0.003 * np.shape(img1)[0]), 1]))
    draw_size2 = int(np.max([np.ceil(0.003 * np.shape(img2)[0]), 1]))
    draw_size3 = int(np.max([0.5 * draw_size1, 1]))
    
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        # color = tuple(np.random.randint(0,255,3).tolist())
        color = tuple([0, 255, 0])
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv.line(img1, (x0,y0), (x1,y1), color,draw_size3)
        img1 = cv.circle(img1,tuple(pt1),draw_size1,color,-1)
        img2 = cv.circle(img2,tuple(pt2),draw_size2,color,-1)
    return img1,img2

def draw_inlier_points(img1,img2,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    color = [0, 0 ,255]
    draw_size1 = int(np.max([np.ceil(0.003 * np.shape(img1)[0]), 1]))
    draw_size2 = int(np.max([np.ceil(0.003 * np.shape(img2)[0]), 1]))

    for pt1,pt2 in zip(pts1,pts2):
        img1 = cv.circle(img1,tuple(pt1),draw_size1,tuple(color),-1)
        img2 = cv.circle(img2,tuple(pt2),draw_size2,tuple(color),-1)
    return img1,img2

def draw_outlier_points(img,pts):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    img = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
    color = [255, 0 ,0]
    draw_size = int(np.max([np.ceil(0.003 * np.shape(img)[0]), 1]))

    for pt in pts:
        img = cv.circle(img,tuple(pt),draw_size,tuple(color),-1)
    return img

In [ ]:
# Configs
matcher_features = "superpoint"
ransacThresh=7
ransacConf=0.99
images = Path("assets")
img1_name = "sacre_coeur1.jpg"
img2_name = "sacre_coeur2.jpg"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

if matcher_features == "superpoint":
    extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
elif matcher_features == "sift":
    extractor = SIFT(max_num_keypoints=2048).eval().to(device)  # load the extractor
img1 = load_image(images / img1_name)
img2 = load_image(images / img2_name)

# image1 = load_image(images / "sacre_coeur2.jpg")
setup = {
            'matcher_model':  LightGlue(features=matcher_features).eval().to(device),
            'matcher_name': "LightGlue",
            }

# Load
matcher = setup['matcher_model']
matcher_name = setup['matcher_name']

# Extract features
feats0 = extractor.extract(img1.to(device))
feats1 = extractor.extract(img2.to(device))

# Match features with matcher
matches01 = matcher({"image0": feats0, "image1": feats1})
feats0, feats1, matches01 = [
    rbd(x) for x in [feats0, feats1, matches01]
]  # remove batch dimension

# Identify matched keypoints
kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

pts1 = np.int32(m_kpts0)
pts2 = np.int32(m_kpts1)

F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC, ransacReprojThreshold=ransacThresh, confidence=ransacConf)


img1 = cv.imread('assets/' + img1_name, cv.IMREAD_GRAYSCALE) #trainimage # left image
img2 = cv.imread('assets/' + img2_name, cv.IMREAD_GRAYSCALE) #trainimage # right image

if mask is None:
    kpts0 = kpts0.int()
    kpts0 = kpts0.numpy()
    kpts1 = kpts1.int()
    kpts1 = kpts1.numpy()
    pts1_outlier = kpts0
    pts2_outlier = kpts1
    pts1 = []
    pts2 = []
    img9 = draw_outlier_points(img1, pts1_outlier)
    img11 = draw_outlier_points(img2, pts2_outlier)    

else:
    pts1_outlier = np.int32(pts1[mask.ravel()==0])
    pts2_outlier = np.int32(pts2[mask.ravel()==0])
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]

    # Find epilines corresponding to points in right image (second image) and
    img3 = draw_outlier_points(img1, pts1_outlier)
    img4 = draw_outlier_points(img2, pts2_outlier)

    # drawing its lines on left image
    lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img3,img4,lines1,pts1,pts2)
    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img7,img8 = drawlines(img4,img3,lines2,pts2,pts1)

    img9,img10 = draw_inlier_points(img5,img6,pts1,pts2)
    img11,img12 = draw_inlier_points(img7,img8,pts2,pts1)


plt.subplot(121),plt.imshow(img9)
plt.subplot(122),plt.imshow(img11)
plt.show()
